# 马尔可夫决策过程

利用强化学习解决实际问题，第一步要做的事情就是把实际问题抽象为马尔可夫决策过程，也就是明确马尔可夫决策过程的各个组成要素，因此马尔可夫决策过程（Markov decision process，MDP）是强化学习的重要概念。。

## 关键概念与定义

### 马尔可夫性质

当且仅当某时刻的状态只取决于上一时刻的状态时，一个随机过程被称为具有马尔可夫性质（Markov property），用公式表示为。也就是说，当前状态是未来的充分统计量，即下一个状态只取决于当前状态，而不会受到过去状态的影响。需要明确的是，具有马尔可夫性并不代表这个随机过程就和历史完全没有关系。因为虽然时刻的状态只与时刻的状态有关，但是时刻的状态其实包含了时刻的状态的信息，通过这种链式的关系，历史的信息被传递到了现在。马尔可夫性可以大大简化运算，因为只要当前状态可知，所有的历史信息都不再需要了，利用当前状态信息就可以决定未来。

举例子, 我们定义一个状态转移的规律如下: 

|         | Winning | Losing | Drawing |
|---------|---------|--------|---------|
| Winning | 0.6     | 0.1    | 0.3     |
| Losing  | 0.3     | 0.4    | 0.3     |
| Drawing | 0.4     | 0.2    | 0.4     |

> 球队本场state到下场state的概率

如果该球队胜利了，那么两场比赛后的状态如下:
Winning = (0.6 * 0.6) + (0.1 * 0.3) + (0.3 * 0.4) = 0.36 + 0.03 + 0.12 = 0.51
Losing = (0.6 * 0.1) + (0.1 * 0.4) + (0.3 * 0.2) = 0.06 + 0.04 + 0.06 = 0.16
Drawing = (0.6 * 0.3) + (0.1 * 0.3) + (0.3 * 0.4) = 0.18 + 0.03 + 0.12 = 0.33

这样我们就完成了一个符合马尔可夫性质的建模。

### 马尔可夫奖励过程（Markov Reward Process, MRP）

描述了在满足马尔可夫性质的序列中，每个状态附带奖励的数学框架。它是马尔可夫决策过程（MDP）的前置概念，去除了“动作”部分，仅关注状态转移的随机性和奖励的期望。它的建模包含了以下的定义:

| **符号**       | **名称**                | **数学定义**                                                                 |
|----------------|-------------------------|----------------------------------------------------------------------------|
| $\mathcal{S}$  | 状态集合                | 所有可能的状态 $s \in \mathcal{S}$                                        |
| $\mathcal{P}$  | 状态转移概率矩阵        | $\mathcal{P}(s' \| s) = \mathbb{P}(S_{t+1}=s' \| S_t = s)$                |
| $\mathcal{R}$  | 奖励函数                | $\mathcal{R}(s) = \mathbb{E}[R_{t+1} \| S_t = s]$                         |
| $\gamma$       | 折扣因子                | $0 \leq \gamma \leq 1$，控制未来奖励的权重                                |


#### 回报
在符合马尔可夫性质的过程中，所有奖励的衰减之和称为回报，现在为我们的球队的结果设置一个奖励矩阵吧:

|         | Winning | Losing | Drawing |
|---------|---------|--------|---------|
| Winning | +10     | -5     | 0       |
| Losing  | +12      | 0      | +2      |
| Drawing | +5      | -3     | 0       |

如此一来, Winning -> Winning -> Losing 的回报就等于: 10 + （-5） = 5

#### 价值函数

在马尔可夫奖励过程中，一个状态的期望回报（即从这个状态出发的未来累积奖励的期望）被称为这个状态的价值（value）。所有状态的价值就组成了价值函数（value function），价值函数的输入为某个状态，输出为这个状态的价值。我们将价值函数写成: 

$$V(s) = \mathbb{E} \left[ \sum_{k=0}^{\infty} \gamma^k R_{t+k+1} \,\bigg|\, S_t = s \right]$$
- **符号说明**  
  - $\gamma$：折扣因子，$0 \leq \gamma < 1$  
  - $R_{t+k+1}$：时间步 $t+k+1$ 的即时奖励

也可以写作**贝尔曼方程**:

$$V(s) = \underbrace{\sum_{s'} P(s' | s) R(s, s')}_{\text{即时奖励期望-r(s)}} + \gamma \underbrace{\sum_{s'} P(s' | s) V(s')}_{\text{未来价值期望}}$$

- **分解含义**  
  - **第一部分**：从状态 \(s\) 转移到所有可能 \(s'\) 的即时奖励期望。  
  - **第二部分**：转移到后续状态后的未来价值期望，按 $\gamma$ 衰减。






In [1]:
import numpy as np

class MarkovRewardProcess:
    def __init__(self, states, P, R, gamma=0.9):
        """
        初始化马尔可夫奖励过程
        :param states: 状态名称列表，例如 ["Winning", "Losing", "Drawing"]
        :param P: 状态转移概率矩阵，shape=(n_states, n_states)
        :param R: 奖励矩阵，R[s][s']表示从状态s转移到s'的即时奖励
        :param gamma: 折扣因子，默认为0.9
        """
        self.states = states
        self.P = np.array(P)
        self.R = np.array(R)
        self.gamma = gamma
        self.n_states = len(states)
        
        # 验证输入合法性
        assert self.P.shape == (self.n_states, self.n_states), "P矩阵维度不匹配"
        assert self.R.shape == (self.n_states, self.n_states), "R矩阵维度不匹配"
        for row in self.P:
            assert np.isclose(row.sum(), 1.0), "状态转移概率之和必须为1"

    def calculate_value_function(self, max_iter=1000, tol=1e-6):
        """
        通过迭代法求解状态价值函数 V
        :param max_iter: 最大迭代次数
        :param tol: 收敛阈值
        :return: 状态价值向量，V[s]对应状态s的价值
        """
        V = np.zeros(self.n_states)  # 初始化价值函数
        
        for _ in range(max_iter):
            V_new = np.zeros(self.n_states)
            for s in range(self.n_states):
                # 计算每个状态的期望奖励和未来价值
                reward = np.sum(self.P[s] * self.R[s])  # 期望即时奖励 R(s)
                future_value = np.sum(self.P[s] * V)       # 未来价值的期望
                V_new[s] = reward + self.gamma * future_value
                
            if np.max(np.abs(V - V_new)) < tol:
                break
            V = V_new.copy()
        
        return V    

states = ["Winning", "Losing", "Drawing"]

# 状态转移概率矩阵 P
# P[s][s']表示从状态s转移到s'的概率
P = [
    [0.5, 0.2, 0.3],  # Winning -> (Winning, Losing, Drawing)
    [0.4, 0.2, 0.4],  # Losing -> (Winning, Losing, Drawing)
    [0.4, 0.2, 0.4]    # Drawing -> (Winning, Losing, Drawing)
]

# 奖励矩阵 R
# R[s][s']表示从状态s转移到s'的即时奖励
R = [
    [10, -5, 0],   # 从Winning转移到各状态的奖励
    [12, 0, 2],   # 从Losing转移到各状态的奖励
    [5,  -3, 0]    # 从Drawing转移到各状态的奖励
]

# 创建MRP并计算价值函数
mrp = MarkovRewardProcess(states, P, R, gamma=0.9)
V_iterative = mrp.calculate_value_function()

# 打印结果
print("状态价值函数 (迭代法):")
for s, name in enumerate(states):
    print(f"  {name}: {V_iterative[s]:.2f}")


状态价值函数 (迭代法):
  Winning: 34.70
  Losing: 36.05
  Drawing: 31.85


### 马尔可夫决策过程 (Markov decision process，MDP)

马尔可夫决策过程是马尔可夫奖励过程（Markov Reward Process, MRP）的升级形式，在原来**状态转换** 和 **奖励**的基础上加上了**行动(Action,A)** 与 **策略(Policy, P)** 的概念。

先来看马尔可夫决策过程的五元组:

- $S$: **状态空间（State Space）**，即智能体可能处于的所有状态的集合。
- $A$: **动作空间（Action Space）**，即智能体可以执行的所有动作的集合。
- $P$: **状态转移概率（Transition Probability）**，$P(s'|s, a)$ 表示在状态 $s$ 采取动作 $a$ 后转移到状态 $s'$ 的概率。
- $R$: **奖励函数（Reward Function）**，$R(s, a)$ 表示在状态 $s$ 采取动作 $a$ 后获得的期望奖励。
- $\gamma$: **折扣因子（Discount Factor）**，$0 \leq \gamma \leq 1$，用于平衡当前奖励和未来奖励的重要性。

#### 状态转移

在 MDP 中，智能体根据某个**策略(Policy)** $\pi(a|s)$ 选择**动作(Action)**，导致状态的变化：
$$
s_t \xrightarrow{a_t} s_{t+1}, \quad R_t = R(s_t, a_t)
$$


#### 例子

![mdp_example](./images/mdp_example.png)

In [2]:
import numpy as np

# 定义状态和动作
S = ["s1", "s2", "s3", "s4", "s5"]  # 状态空间
A = ["保持s1", "前往s1", "前往s2", "前往s3", "前往s4", "前往s5", "概率前往"]  # 动作空间

def join(state, action):
    return f"{state}-{action}"

# 状态转移概率 P(s' | s, a)
P = {
    "s1-保持s1-s1": 1.0,
    "s1-前往s2-s2": 1.0,
    "s2-前往s1-s1": 1.0,
    "s2-前往s3-s3": 1.0,
    "s3-前往s4-s4": 1.0,
    "s3-前往s5-s5": 1.0,
    "s4-前往s5-s5": 1.0,
    "s4-概率前往-s2": 0.2,
    "s4-概率前往-s3": 0.4,
    "s4-概率前往-s4": 0.4,
}

# 奖励函数 R(s, a)
R = {
    "s1-保持s1": -1,
    "s1-前往s2": 0,
    "s2-前往s1": -1,
    "s2-前往s3": -2,
    "s3-前往s4": -2,
    "s3-前往s5": 0,
    "s4-前往s5": 10,
    "s4-概率前往": 1,
}

gamma = 0.5  # 折扣因子

# 策略1: 随机策略
Pi_1 = {
    "s1-保持s1": 0.5,
    "s1-前往s2": 0.5,
    "s2-前往s1": 0.5,
    "s2-前往s3": 0.5,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.5,
    "s4-概率前往": 0.5,
}

Pi_2 = {
    "s1-保持s1": 0.6,
    "s1-前往s2": 0.4,
    "s2-前往s1": 0.3,
    "s2-前往s3": 0.7,
    "s3-前往s4": 0.5,
    "s3-前往s5": 0.5,
    "s4-前往s5": 0.1,
    "s4-概率前往": 0.9,
}

# 计算状态价值函数 V_pi(s)
def evaluate_policy(Pi, theta=1e-6, max_iters=1000):
    V = {s: 0 for s in S}  # 初始化所有状态的价值为0
    
    for _ in range(max_iters):
        delta = 0
        V_new = V.copy()
        
        for s in S:
            v = 0
            for a in A:
                key_sa = join(s, a)
                if key_sa in Pi:
                    action_prob = Pi[key_sa]  # 策略下采取动作 a 的概率
                    reward = R.get(key_sa, 0)  # 立即奖励 R(s,a)
                    
                    expected_value = 0
                    for s_next in S:
                        key_sas = f"{key_sa}-{s_next}"
                        transition_prob = P.get(key_sas, 0)
                        expected_value += transition_prob * V[s_next]
                    
                    v += action_prob * (reward + gamma * expected_value)
            
            V_new[s] = v
            delta = max(delta, abs(V_new[s] - V[s]))
        
        V = V_new
        if delta < theta:
            break
    
    return V

# 计算动作价值函数 Q_pi(s, a)
def compute_q_values(V, Pi):
    Q = np.zeros((len(S), len(A)))  # 创建Q值矩阵，行为状态，列为动作
    state_idx = {s: i for i, s in enumerate(S)}
    action_idx = {a: i for i, a in enumerate(A)}
    
    for s in S:
        for a in A:
            key_sa = join(s, a)
            reward = R.get(key_sa, 0)
            expected_value = 0
            
            for s_next in S:
                key_sas = f"{key_sa}-{s_next}"
                transition_prob = P.get(key_sas, 0)
                expected_value += transition_prob * V[s_next]
            
            if key_sa in Pi:
                Q[state_idx[s], action_idx[a]] = reward + gamma * expected_value
    
    return Q

# 计算状态价值函数 V_pi(s) 和 动作价值函数 Q_pi(s,a)
V_pi_1 = evaluate_policy(Pi_1)
Q_pi_1 = compute_q_values(V_pi_1, Pi_1)

V_pi_2 = evaluate_policy(Pi_2)
Q_pi_2 = compute_q_values(V_pi_2, Pi_2)


print("\n策略1动作价值函数 Q_pi_1(s, a) 矩阵:")
print("\t" + "\t".join(A))
for i, s in enumerate(S):
    row_values = "\t".join(f"{val:.2f}" for val in Q_pi_1[i])
    print(f"{s}\t{row_values}")

print("\n策略2动作价值函数 Q_pi_2(s, a) 矩阵:")
print("\t" + "\t".join(A))
for i, s in enumerate(S):
    row_values = "\t".join(f"{val:.2f}" for val in Q_pi_2[i])
    print(f"{s}\t{row_values}")



策略1动作价值函数 Q_pi_1(s, a) 矩阵:
	保持s1	前往s1	前往s2	前往s3	前往s4	前往s5	概率前往
s1	-1.61	0.00	-0.84	0.00	0.00	0.00	0.00
s2	0.00	-1.61	0.00	-1.74	0.00	0.00	0.00
s3	0.00	0.00	0.00	0.00	1.04	0.00	0.00
s4	0.00	0.00	0.00	0.00	0.00	10.00	2.15
s5	0.00	0.00	0.00	0.00	0.00	0.00	0.00

策略2动作价值函数 Q_pi_2(s, a) 矩阵:
	保持s1	前往s1	前往s2	前往s3	前往s4	前往s5	概率前往
s1	-1.73	0.00	-1.05	0.00	0.00	0.00	0.00
s2	0.00	-1.73	0.00	-2.25	0.00	0.00	0.00
s3	0.00	0.00	0.00	0.00	-1.01	0.00	0.00
s4	0.00	0.00	0.00	0.00	0.00	10.00	1.08
s5	0.00	0.00	0.00	0.00	0.00	0.00	0.00


## 总结

届此，我们发现**强化学习中的环境就是一个马尔可夫决策过程**，而强化学习的目标就是找到马尔可夫决策过程中的最优策略。